In [1]:
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob

cases = {
    "vehicle-to-vehicle": {
        "real_txt": "results_vv/test0418_Tucsan_Received.txt",
        "virt_txt": "results_vv/test0418_sedan_to_vehicle.txt",
        "timestamps": [
            "results_vv/Timestamp/virtual_vehicle_scenario_with_intelligence/*.json",
            "results_vv/Timestamp/virtual_vehicle_scenario_without_intelligence/*.json"
        ]
    },
    "vehicle-to-pedestrian": {
        "real_txt": "results_vp/test0418_Tucsan_Received.txt",
        "virt_txt": "results_vp/test0418_pedestrian_to_vehicle.txt",
        "timestamps": [
            "results_vp/Timestamp/virtual_pedestrian_scenario_with_intelligence/*.json",
            "results_vp/Timestamp/virtual_pedestrian_scenario_without_intelligence/*.json"
        ]
    }
}

def extract_latency_jitter(real_file, virt_file, timestamp_globs):
    latency_combined = []
    jitter_combined = []

    with open(real_file, "r") as f:
        real_lines = f.readlines()
    with open(virt_file, "r") as f:
        virt_lines = f.readlines()

    print(f"Total real lines: {len(real_lines)}")
    print(f"Total virtual lines: {len(virt_lines)}")

    for glob_pattern in timestamp_globs:
        for ts_file in glob(glob_pattern):
            with open(ts_file, "r") as f:
                try:
                    data = json.load(f)
                    entries = data.get("DataEntries", [])
                    if not entries:
                        continue
                    start = float(entries[0]["Timestamp"])
                    end = float(entries[-1]["Timestamp"])
                    print(f"Processing {ts_file}")
                except:
                    continue

            lat_real, recv_real = [], []
            lat_virt, recv_virt = [], []

            for line in real_lines:
                try:
                    rec = json.loads(line)
                    t_recv = rec.get("timeStamp")
                    t_sent = rec.get("t_veh_sent")
                    if t_recv is None or t_sent is None:
                        continue
                    if start <= t_recv <= end and t_sent > 0:
                        latency = t_recv - t_sent
                        if 0 < latency <= 0.5:
                            lat_real.append(latency)
                            recv_real.append(t_recv)
                except:
                    continue

            for line in virt_lines:
                try:
                    rec = json.loads(line)
                    t_recv = rec.get("timeStamp")
                    t_sent = rec.get("t_vir_obj_sent")
                    if t_recv is None or t_sent is None:
                        continue
                    if start <= t_recv <= end and t_sent > 0:
                        latency = t_recv - t_sent
                        if 0 < latency <= 0.5:
                            lat_virt.append(latency)
                            recv_virt.append(t_recv)
                except:
                    continue

            latency_combined.extend(lat_real + lat_virt)

            if len(recv_real) > 1:
                jitter_real = np.diff(sorted(recv_real))
                jitter_real = jitter_real[(jitter_real > 0) & (jitter_real <= 0.8)]
                jitter_combined.extend(jitter_real.tolist())

            if len(recv_virt) > 1:
                jitter_virt = np.diff(sorted(recv_virt))
                jitter_virt = jitter_virt[(jitter_virt > 0) & (jitter_virt <= 0.8)]
                jitter_combined.extend(jitter_virt.tolist())

    return latency_combined, jitter_combined

results = {}
summaries = []

for label, cfg in cases.items():
    latency, jitter = extract_latency_jitter(cfg["real_txt"], cfg["virt_txt"], cfg["timestamps"])
    if not latency:
        print(f"No valid latency for {label}")
        continue
    results[label] = {"latency": latency, "jitter": jitter}
    summaries.append({
        "Case": label,
        "Mean Latency": np.mean(latency),
        "Std Latency": np.std(latency),
        "Min Latency": np.min(latency),
        "Max Latency": np.max(latency),
        "Mean Jitter": np.mean(jitter),
        "Std Jitter": np.std(jitter),
        "Min Jitter": np.min(jitter),
        "Max Jitter": np.max(jitter)
    })

summary_df = pd.DataFrame(summaries)
summary_df.to_csv("combined_latency_jitter_summary.csv", index=False)

if results:
    plt.figure(figsize=(6, 6))
    plt.boxplot([results[k]["latency"] for k in results], labels=results.keys(), showfliers=True)
    plt.title("Latency")
    plt.ylabel("Network measurements (s)")
    plt.ylim(0, 0.5)
    plt.grid(True, axis='y')
    plt.tight_layout()
    plt.savefig("latency_boxplot_combined.png")
    plt.close()

    plt.figure(figsize=(6, 6))
    plt.boxplot([results[k]["jitter"] for k in results], labels=results.keys(), showfliers=True)
    plt.title("Jitter")
    plt.ylabel("Network measurements (s)")
    plt.ylim(0, 0.8)
    plt.grid(True, axis='y')
    plt.tight_layout()
    plt.savefig("jitter_boxplot_combined.png")
    plt.close()

    print("Final combined boxplots saved.")
else:
    print("No valid data found for any case.")

Total real lines: 72878
Total virtual lines: 317311
Processing results_vv/Timestamp/virtual_vehicle_scenario_with_intelligence\vv10.json
Processing results_vv/Timestamp/virtual_vehicle_scenario_with_intelligence\vv11.json
Processing results_vv/Timestamp/virtual_vehicle_scenario_with_intelligence\vv12.json
Processing results_vv/Timestamp/virtual_vehicle_scenario_with_intelligence\vv13noss.json
Processing results_vv/Timestamp/virtual_vehicle_scenario_with_intelligence\vv14noss.json
Processing results_vv/Timestamp/virtual_vehicle_scenario_with_intelligence\vv16.json
Processing results_vv/Timestamp/virtual_vehicle_scenario_with_intelligence\vv1good.json
Processing results_vv/Timestamp/virtual_vehicle_scenario_with_intelligence\vv4.json
Processing results_vv/Timestamp/virtual_vehicle_scenario_with_intelligence\vv7.json
Processing results_vv/Timestamp/virtual_vehicle_scenario_with_intelligence\vv8ss안들어감.json
Processing results_vv/Timestamp/virtual_vehicle_scenario_without_intelligence\vvn1.j